# Report

?? Initial suggested structure

 1. Data Exploration - Explore the data and show you understand its structure and relations
 2. Data Validation - Check the quality of the data. Is it complete? Are there obvious errors?
 3. Data Visualisation - Gain an overall understanding of the data with visualisations
 4. Data Analysis = Set some questions and use the data to answer them
 5. Data Augmentation - Add new data from another source to bring new insights to the data you already have